In [1]:
# https://blog.naver.com/ostin1038/222116828334

In [ ]:
# 나오기 어렵다 fp growth vs  

In [2]:
_ = """

1. 지지도(Support)
   전체 거래 항목 중 A, B를 동시에 포함하여 거래하는 비율

   지지도 s(A→B) = P(A∩B)  :  A와 B가 동시에 포함된 거래 수 / 전체 거래 수
   
   : 거래 빈도

2. 신뢰도(Confidence)

   상품 A를 포함하는 거래 중 A와 B가 동시에 거래되는 비중
   상품 A를 구매 했을 때 상품 B를 구매할 확률이 어느정도 되는지를 확인합니다.


   신뢰도 c(A→B) =  P(A∩B) / P(A)  :  A와 B가 동시에 포함된 거래 수 / A가 포함된 거래 수

   : 연관성

3. 향상도(Lift)

  상품 A의 거래 중 항목 B가 포함된 거래의 비율 / 전체 상품 거래 중 상품 B가 거래된 비율
  (A가 주어지지 않았을 때 B의 확률 대비 A가 주어졌을 때 B의 확률 증가 비율)

  향상도   l(A→B) = P(A∩B) / P(A)*P(B)  = P (B|A) / P (B)

  :  A와 B가 동시에 일어난 횟수 / A, B가 독립된 사건일 때 A,B가 동시에 일어날 확률

"""

In [2]:
import pandas as pd
import numpy as np

In [137]:
EXAMPLE = pd.read_csv('data/instacart_example.csv')
PRODUCT = pd.read_csv('data/instacart_product.csv', encoding='cp949')
DEPARTMENTS = pd.read_csv('data/instacart_departments.csv')

In [4]:
EXAMPLE.head()

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,department_id
0,2539329,1,1,2,8,NaN,196,1,0,7
1,2539329,1,1,2,8,NaN,14084,2,0,16
2,2539329,1,1,2,8,NaN,12427,3,0,19
3,2539329,1,1,2,8,NaN,26088,4,0,19
4,2539329,1,1,2,8,NaN,26405,5,0,17


In [5]:
PRODUCT.head()

,product_id,product_name,department_id
0,1,Chocolate Sandwich Cookies,19.0
1,2,All-Seasons Salt,13.0
2,3,Robust Golden Unsweetened Oolong Tea,7.0
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,1.0
4,5,Green Chile Anytime Sauce,13.0


In [6]:
DEPARTMENTS.head()

,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol


In [7]:
DEPARTMENTS.shape

(21, 2)

In [8]:
EXAMPLE[EXAMPLE.product_id == 14084]

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,department_id
1,2539329,1,1,2,8,NaN,14084,2,0,16
17039,3241517,140,14,4,16,7.0,14084,25,0,16
45037,1204424,293,1,0,20,NaN,14084,2,0,16
45054,2748748,293,2,1,20,15.0,14084,2,1,16
45095,1216787,293,5,4,18,17.0,14084,10,1,16
45098,1249341,293,6,3,17,30.0,14084,1,1,16
45123,1055717,293,7,4,18,29.0,14084,1,1,16
45145,666649,293,8,4,17,7.0,14084,5,1,16
45562,512481,297,3,2,15,30.0,14084,2,0,16
45593,3068717,297,10,1,18,26.0,14084,3,1,16


In [9]:
PRODUCT[PRODUCT.product_id==14084]

,product_id,product_name,department_id
14083,14084,Organic Unsweetened Vanilla Almond Milk,16.0


In [10]:
D = pd.merge(
    pd.merge(EXAMPLE, PRODUCT[['product_id','product_name']], on='product_id', how='left'),
    DEPARTMENTS,on='department_id', how='left'
    )

In [11]:
D.head()

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,department_id,product_name,department
0,2539329,1,1,2,8,NaN,196,1,0,7,Soda,beverages
1,2539329,1,1,2,8,NaN,14084,2,0,16,Organic Unsweetened Vanilla Almond Milk,dairy eggs
2,2539329,1,1,2,8,NaN,12427,3,0,19,Original Beef Jerky,snacks
3,2539329,1,1,2,8,NaN,26088,4,0,19,Aged White Cheddar Popcorn,snacks
4,2539329,1,1,2,8,NaN,26405,5,0,17,XL Pick-A-Size Paper Towel Rolls,household


In [12]:
D.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 77425 entries, 0 to 77424
Data columns (total 12 columns):
order_id                  77425 non-null int64
user_id                   77425 non-null int64
order_number              77425 non-null int64
order_dow                 77425 non-null int64
order_hour_of_day         77425 non-null int64
days_since_prior_order    72525 non-null float64
product_id                77425 non-null int64
add_to_cart_order         77425 non-null int64
reordered                 77425 non-null int64
department_id             77425 non-null int64
product_name              77425 non-null object
department                77425 non-null object
dtypes: float64(1), int64(9), object(2)
memory usage: 7.7+ MB


In [13]:
D.describe()

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,department_id
count,7.742500e+04,77425.000000,77425.000000,77425.000000,77425.000000,72525.00000,77425.000000,77425.000000,77425.000000,77425.000000
mean,1.696186e+06,260.848576,16.878579,2.759755,13.520052,11.08080,25815.593323,8.360452,0.583687,10.010216
std,9.880130e+05,140.175786,17.531267,2.087571,4.273775,8.82473,14216.871937,7.193211,0.492950,6.247321
min,4.000000e+01,1.000000,1.000000,0.000000,0.000000,0.00000,1.000000,1.000000,0.000000,1.000000
25%,8.567530e+05,149.000000,5.000000,1.000000,10.000000,5.00000,13535.000000,3.000000,0.000000,4.000000
50%,1.671477e+06,262.000000,11.000000,3.000000,13.000000,7.00000,25740.000000,6.000000,1.000000,9.000000
75%,2.564286e+06,381.000000,23.000000,5.000000,17.000000,15.00000,38293.000000,11.000000,1.000000,16.000000
max,3.420572e+06,500.000000,99.000000,6.000000,23.000000,30.00000,49683.000000,89.000000,1.000000,21.000000


In [14]:
# Q1: 현재 구매데이터 셋을 바탕으로 요일별 방문자수를 확인하고
#  (이때 방문자수는 user_id가 아닌 order_id를 기준으로 확인한다)
#  가장 많은 방문자를 기록한 요일을 먼저 확인한다.
#  그후 해당 요일에 가장 많이 팔린 상품ID 및 판매 수를 확인하라

In [15]:
EXAMPLE.groupby(by='order_id').max()[['order_dow']]

,order_dow
order_id,
40,1
1483,3
2199,4
2593,6
3449,1
4163,2
4307,3
4595,3
4890,2


In [16]:
D11 = EXAMPLE.groupby(by='order_id').max()[['order_dow']]

In [17]:
D11.reset_index().groupby(by='order_dow').agg('count').sort_values('order_id',ascending=False)

,order_id
order_dow,
1,1424
0,1248
2,1089
5,1082
3,968
4,924
6,907


In [18]:
D[D.order_dow == 1].groupby(by=['product_id','product_name']).agg('count').sort_values('order_id', ascending=False)

,,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,add_to_cart_order,reordered,department_id,department
product_id,product_name,,,,,,,,,,
13176,Bag of Organic Bananas,183,183,183,183,183,174,183,183,183,183
24852,Banana,156,156,156,156,156,145,156,156,156,156
21137,Organic Strawberries,146,146,146,146,146,138,146,146,146,146
47209,Organic Hass Avocado,129,129,129,129,129,122,129,129,129,129
21903,Organic Baby Spinach,111,111,111,111,111,103,111,111,111,111
47766,Organic Avocado,90,90,90,90,90,87,90,90,90,90
45007,Organic Zucchini,65,65,65,65,65,62,65,65,65,65
39275,Organic Blueberries,60,60,60,60,60,55,60,60,60,60
27966,Organic Raspberries,59,59,59,59,59,58,59,59,59,59


In [3]:
# Q2
_ = """
Q2: Q1을 통해 확인한 요일의 판매 데이터만 추출해 연관성 분석을 진행하고자 한다. 
    주문 내역(order_id) 내 상품 코너(department) 간 연관성을 확인하고자 하며 
    선행, 후행 단일 항목으로 구성된 규칙을 추출하고자 한다.
   이때 제시된 연관규칙 중 babies 코너와 Lift 값이 가장 높게 나온 코너는 무엇인가? 
   (Min Supprot: 0.03 confidence: 0.85 나머지 default 값 ) 
"""

In [44]:
D[D.order_dow == 1].department.value_counts().head()

produce       4099
dairy eggs    2440
snacks        1418
beverages     1252
frozen         880
Name: department, dtype: int64

In [45]:
D[D.order_dow == 1].groupby(by='order_id').department.agg(lambda x: list(x))

order_id
40               [dairy eggs, frozen, beverages, dairy eggs]
3449       [meat seafood, beverages, produce, beverages, ...
6524       [pantry, pantry, produce, beverages, beverages...
10985                     [beverages, beverages, dairy eggs]
14197       [produce, snacks, snacks, canned goods, produce]
14400                              [snacks, snacks, alcohol]
19190      [canned goods, frozen, frozen, frozen, frozen,...
19256      [dairy eggs, bakery, dry goods pasta, pantry, ...
19682      [produce, produce, produce, snacks, snacks, pa...
29171                     [frozen, produce, produce, pantry]
30057      [deli, snacks, snacks, snacks, produce, produc...
30855      [deli, pets, beverages, dairy eggs, dry goods ...
31801      [frozen, snacks, produce, meat seafood, dairy ...
33104      [babies, breakfast, breakfast, breakfast, babi...
34164      [pets, beverages, dairy eggs, dairy eggs, dry ...
46306      [pantry, pantry, pantry, dry goods pasta, prod...
46584          

In [54]:
D[D.order_dow == 1].groupby(by='order_id').department.agg(lambda x: list(set(x))).values

array([list(['dairy eggs', 'frozen', 'beverages']),
       list(['produce', 'beverages', 'meat seafood']),
       list(['produce', 'dairy eggs', 'pantry', 'beverages', 'meat seafood']),
       ...,
       list(['babies', 'produce', 'bakery', 'dairy eggs', 'dry goods pasta', 'meat seafood']),
       list(['produce', 'dairy eggs', 'deli', 'frozen', 'meat seafood']),
       list(['produce', 'bakery', 'dairy eggs', 'snacks', 'canned goods', 'pantry', 'beverages'])],
      dtype=object)

In [55]:
records = D[D.order_dow == 1].groupby(by='order_id').department.agg(lambda x: list(set(x))).values

In [22]:
from apyori import apriori

In [56]:
association_rules = apriori(records, min_support=0.03, min_confidence=0.85)

In [57]:
# association_results = list(association_rules)
association_results = list(association_rules)

In [58]:
association_results

[RelationRecord(items=frozenset({'dairy eggs', 'babies'}), support=0.047050561797752806, ordered_statistics=[OrderedStatistic(items_base=frozenset({'babies'}), items_add=frozenset({'dairy eggs'}), confidence=0.9178082191780821, lift=1.37429958371145)]),
 RelationRecord(items=frozenset({'babies', 'produce'}), support=0.04564606741573034, ordered_statistics=[OrderedStatistic(items_base=frozenset({'babies'}), items_add=frozenset({'produce'}), confidence=0.8904109589041096, lift=1.2334097329566653)]),
 RelationRecord(items=frozenset({'produce', 'canned goods'}), support=0.17907303370786518, ordered_statistics=[OrderedStatistic(items_base=frozenset({'canned goods'}), items_add=frozenset({'produce'}), confidence=0.9172661870503597, lift=1.2706099711670351)]),
 RelationRecord(items=frozenset({'produce', 'deli'}), support=0.19241573033707865, ordered_statistics=[OrderedStatistic(items_base=frozenset({'deli'}), items_add=frozenset({'produce'}), confidence=0.8698412698412699, lift=1.204916311531

In [90]:
for x in association_results:

    if 'babies' in x[0]:
        print("item", x[0]) # items
        print("\tsupport", x[1]) # support
        print("\tlift", x[2][0].lift)  # lify

item frozenset({'dairy eggs', 'babies'})
	support 0.047050561797752806
	lift 1.37429958371145
item frozenset({'babies', 'produce'})
	support 0.04564606741573034
	lift 1.2334097329566653
item frozenset({'dairy eggs', 'babies', 'produce'})
	support 0.042134831460674156
	lift 1.240490156222777


In [4]:
# Q3
_ = """
Q3: 구매한 상품이 재구매 상품인지 분류하는 Decision Tree 모델을 만들고자한다. 
먼저 데이터 분할은 고객(user_id)별 가장 마지막 주문 내역의 구매 상품들을 테스트셋으로 나머지는 트레인셋으로 구분한다.

다음으로 트레인셋 DB를 대상으로 상품(product_id)별 재구매 비중을 확인하여 
테스트셋, 트레인셋 동일하게 상품의 재구매 비율(reordered_rate) 변수로 추가한다. 
(테스트셋 join 후 재구매 비율의 결측치는 0으로 변환한다)

트레인셋 총 72,155행, 테스트셋 총 5,270셋
독립변수: 구매 요일(order_dow), 구매 시간(order_hour_of_day), 이전 구매일(days_since_prior), 
상품의 재구매 비율(reordered_rate) 
종속변수: reordered

트레인셋에 맞춰 모델을 만들어 테스트셋으로 예측을 한 후 재구매 예측에 대한 재현율(recall)을 구하여라 
(Criterion: Gini, Splitter: Best, Max Depth:6, Seed: 13, Max Leaf Nodes: 25, 나머지 default) 

ex) 0.48(둘째자리만 나타냄, 셋째자리 버림)
"""

In [92]:
D.head()

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,department_id,product_name,department
0,2539329,1,1,2,8,NaN,196,1,0,7,Soda,beverages
1,2539329,1,1,2,8,NaN,14084,2,0,16,Organic Unsweetened Vanilla Almond Milk,dairy eggs
2,2539329,1,1,2,8,NaN,12427,3,0,19,Original Beef Jerky,snacks
3,2539329,1,1,2,8,NaN,26088,4,0,19,Aged White Cheddar Popcorn,snacks
4,2539329,1,1,2,8,NaN,26405,5,0,17,XL Pick-A-Size Paper Towel Rolls,household


In [98]:
last_order_id = D.groupby(by='user_id').order_id.max()

In [104]:
D[D.order_id.isin(last_order_id)].order_id.nunique()

500

In [105]:
D[~D.order_id.isin(last_order_id)].order_id.nunique()

7142

In [108]:
D['last'] = D.order_id.isin(last_order_id)

In [107]:
D.head()

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,department_id,product_name,department,last
0,2539329,1,1,2,8,NaN,196,1,0,7,Soda,beverages,False
1,2539329,1,1,2,8,NaN,14084,2,0,16,Organic Unsweetened Vanilla Almond Milk,dairy eggs,False
2,2539329,1,1,2,8,NaN,12427,3,0,19,Original Beef Jerky,snacks,False
3,2539329,1,1,2,8,NaN,26088,4,0,19,Aged White Cheddar Popcorn,snacks,False
4,2539329,1,1,2,8,NaN,26405,5,0,17,XL Pick-A-Size Paper Towel Rolls,household,False


In [109]:
pd.crosstab( D['product_id'], D['reordered'])


reordered,0,1
product_id,,
1,1,1
9,1,0
10,5,8
13,1,0
23,2,0
25,2,5
29,1,1
30,1,0
34,8,11


In [111]:
D31 = D.groupby(by='product_id').reordered.agg(['count',np.sum])
D31['reorder_rate'] = D31['sum']/D31['count']
D31.head()

,count,sum,reorder_rate
product_id,,,
1,2,1,0.500000
9,1,0,0.000000
10,13,8,0.615385
13,1,0,0.000000
23,2,0,0.000000


In [113]:
D32 = pd.merge(D, D31, how='left', on='product_id')
D32.head()

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,department_id,product_name,department,last,count,sum,reorder_rate
0,2539329,1,1,2,8,NaN,196,1,0,7,Soda,beverages,False,102,79,0.774510
1,2539329,1,1,2,8,NaN,14084,2,0,16,Organic Unsweetened Vanilla Almond Milk,dairy eggs,False,26,16,0.615385
2,2539329,1,1,2,8,NaN,12427,3,0,19,Original Beef Jerky,snacks,False,30,23,0.766667
3,2539329,1,1,2,8,NaN,26088,4,0,19,Aged White Cheddar Popcorn,snacks,False,6,1,0.166667
4,2539329,1,1,2,8,NaN,26405,5,0,17,XL Pick-A-Size Paper Towel Rolls,household,False,7,2,0.285714


In [121]:
D33 = D32[['order_dow', 'order_hour_of_day', 'days_since_prior_order', 'reorder_rate', 'reordered', 'last']].copy()
D33.head()

,order_dow,order_hour_of_day,days_since_prior_order,reorder_rate,reordered,last
0,2,8,NaN,0.774510,0,False
1,2,8,NaN,0.615385,0,False
2,2,8,NaN,0.766667,0,False
3,2,8,NaN,0.166667,0,False
4,2,8,NaN,0.285714,0,False


In [120]:
### days_since_prior_order 에 대한 처리 필요.. 
# D.fillna(D.mean)

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,department_id,product_name,department,last
0,2539329,1,1,2,8,<bound method DataFrame.mean of order_i...,196,1,0,7,Soda,beverages,False
1,2539329,1,1,2,8,<bound method DataFrame.mean of order_i...,14084,2,0,16,Organic Unsweetened Vanilla Almond Milk,dairy eggs,False
2,2539329,1,1,2,8,<bound method DataFrame.mean of order_i...,12427,3,0,19,Original Beef Jerky,snacks,False
3,2539329,1,1,2,8,<bound method DataFrame.mean of order_i...,26088,4,0,19,Aged White Cheddar Popcorn,snacks,False
4,2539329,1,1,2,8,<bound method DataFrame.mean of order_i...,26405,5,0,17,XL Pick-A-Size Paper Towel Rolls,household,False
5,2398795,1,2,3,7,15,196,1,1,7,Soda,beverages,False
6,2398795,1,2,3,7,15,10258,2,0,19,Pistachios,snacks,False
7,2398795,1,2,3,7,15,12427,3,1,19,Original Beef Jerky,snacks,False
8,2398795,1,2,3,7,15,13176,4,0,4,Bag of Organic Bananas,produce,False
9,2398795,1,2,3,7,15,26088,5,1,19,Aged White Cheddar Popcorn,snacks,False


In [122]:
from sklearn.impute import SimpleImputer

In [124]:
imp = SimpleImputer(strategy="mean")

In [142]:
from  sklearn.tree import DecisionTreeClassifier

In [143]:
# (Criterion: Gini, Splitter: Best, Max Depth:6, Seed: 13, Max Leaf Nodes: 25, 나머지 default) 
decision_clf = DecisionTreeClassifier(criterion="gini", splitter="best", max_depth=6, max_leaf_nodes=25, random_state=13)

In [129]:
from sklearn.pipeline import Pipeline

In [144]:
pipe = Pipeline([('imp', imp), ('reg', decision_clf)])

In [145]:
test_X = D33[D33['last']==True][['order_dow', 'order_hour_of_day', 'days_since_prior_order', 'reorder_rate']]
test_y = D33[D33['last']==True].reordered
train_X = D33[D33['last']==False][['order_dow', 'order_hour_of_day', 'days_since_prior_order', 'reorder_rate']]
train_y = D33[D33['last']==False].reordered

In [146]:
test_X.shape, test_y.shape, train_X.shape, train_y.shape

((4812, 4), (4812,), (72613, 4), (72613,))

In [147]:
pipe.fit(train_X, train_y)

Pipeline(memory=None,
     steps=[('imp', SimpleImputer(copy=True, fill_value=None, missing_values=nan, strategy='mean',
       verbose=0)), ('reg', DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=6,
            max_features=None, max_leaf_nodes=25,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=13,
            splitter='best'))])

In [149]:
predict = pipe.predict(test_X)

In [150]:
from sklearn.metrics import recall_score

In [151]:
recall_score(test_y, predict)

0.8708885616102111